In [ ]:
# Rod (1D)

# Following this video: https://www.youtube.com/watch?v=CXOrkQs4WYo

import numpy as np
import matplotlib.pyplot as plt

alpha = 110
length = 50 # length of the rod, 50 mm
time = 1 # total time, 10 s
nodes = 20 # number of nodes - 1

dx = length / nodes # distance between nodes
dt = 0.2 * dx**2 / alpha # time step, must be less than dx^2 / (2 * alpha)

x_train_1d = np.linspace(0, length, nodes)
t_train_1d = np.linspace(0, time, int(time / dt))

u = np.zeros(nodes) + 20 # middle of the rod is 20 degrees
u[0] = 100 # left end of the rod is 100 degrees
u[-1] = 0 # right end of the rod is 0 degrees

heat_data_1d = np.zeros((int(time / dt), nodes))

for counter in range(heat_data_1d.shape[0]):
    w = u.copy()
    heat_data_1d[counter, :] = u.copy()

    for i in range(1, nodes - 1):
        u[i] = dt * alpha * (w[i - 1] - 2 * w[i] + w[i + 1]) / dx**2 + w[i]
    
    

    # print(f't: {counter * dt:.3f} s, Ave temp: {np.mean(u):.2f} C')

# heat_data_scaled = (heat_data_1d - np.min(heat_data_1d))
# heat_data_scaled = heat_data_scaled / np.max(heat_data_scaled)

In [ ]:
fig, axis = plt.subplots()
pcm = axis.pcolormesh(x_train_1d, t_train_1d, heat_data_1d, cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature')
# axis.set_title(f't: {450*dt:.3f} s')
plt.xlabel('x (mm)')
plt.ylabel('t (s)')
plt.show()

In [ ]:
import os
import sys

parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

In [ ]:
from deep_learning import deep_network_core as core, utils
import torch
import torch.nn as nn
from torch.autograd import grad as autograd

In [ ]:
# torch.manual_seed(1234)
# np.random.seed(1234)

Tr_1d, Xr_1d = np.meshgrid(t_train_1d, x_train_1d)

inpt_1d = np.column_stack((Tr_1d.T.ravel(), Xr_1d.T.ravel()))
oupt_1d = heat_data_1d.ravel()


In [ ]:
inpt_1d.shape

In [ ]:
class MSE_Loss(core.LOSS):
    def __init__(self):
        self.loss = nn.MSELoss()
        
    def __call__(self, target, result, model):
        return self.loss(target, result)

class Huber_Loss(core.LOSS):
    def __init__(self):
        self.loss = nn.HuberLoss()
        
    def __call__(self, target, result, model):
        return self.loss(target, result)

class PHYSICS_Loss(core.LOSS):
    # def __init__(self, bounds):
    #     self.bounds = bounds
    
    def __call__(self, target, result, model):
        x = torch.empty((100, 1)).uniform_(0, length).requires_grad_(True)
        t = torch.empty((100, 1)).uniform_(0, time).requires_grad_(True)
        inp = torch.cat((x, t), axis=1)
        zs = model(inp)
        pde = utils.dy_dt(zs, t) - utils.laplacian_1d(zs, x) # dz/dt - ddz/dxx
        return torch.mean(pde**2)

In [ ]:
network_pinn_1d = core.PINN(2, 1, 128, 8, [(1, MSE_Loss()), (1, PHYSICS_Loss())])

In [ ]:
print("Training PINN")
network_pinn_1d.fit(inpt_1d, oupt_1d, lr=1e-4, epochs=6000)

# Try a learning rate scheduler
# Try Huber loss (a feature in PyTorch)
# See if I can find someone else's PINN on the Heat Eqn

In [ ]:
predicted = network_pinn_1d.predict(inpt_1d).reshape(heat_data_1d.shape)

fig, axis = plt.subplots()
pcm = axis.pcolormesh(x_train_1d, t_train_1d, predicted, cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
# axis.set_title(f't: {450*dt:.3f} s')
plt.xlabel('x (mm)')
plt.ylabel('t (s)')
plt.show()

In [ ]:
network_dgm = core.DGM(2, 1, 128, 4, [(1, MSE_Loss()), (1, PHYSICS_Loss())])
print("Training DGM")
network_dgm.fit(inpt_1d, oupt_1d, lr=1e-2, epochs=200)

In [ ]:
predicted = network_dgm.predict(inpt_1d).reshape(heat_data_1d.shape)

fig, axis = plt.subplots()
pcm = axis.pcolormesh(x_train_1d, t_train_1d, predicted, cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
# axis.set_title(f't: {450*dt:.3f} s')
plt.xlabel('x (mm)')
plt.ylabel('t (s)')
plt.show()